# Analyzing the Metadata of Publications in the Open Research Knowledge Graph 
This Jupyter notebook contains different analyses on the metadata of publications stored in the Open Research Knowledge Graph [ORKG](https://www.orkg.org/orkg/).

## Loading Data from the ORKG SPAQRL endpoint

In [86]:
%matplotlib widget
from SPARQLWrapper import SPARQLWrapper, CSV
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

ENDPOINT_URL = "https://www.orkg.org/orkg/triplestore"

PREFIXES =  """
            PREFIX orkgr: <http://orkg.org/orkg/resource/>
            PREFIX orkgc: <http://orkg.org/orkg/class/>
            PREFIX orkgp: <http://orkg.org/orkg/predicate/>
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            """

query = """
        SELECT ?paper, ?field_label, ?DOI, ?title, ?name, ?id, ?month_number, ?year_number, ?paper_url, ?venue_label
WHERE {
  ?paper a orkgc:Paper.
  OPTIONAL{?paper rdfs:label ?title.}
  OPTIONAL{?paper orkgp:P26 ?DOI.}
  OPTIONAL{?paper orkgp:P30 ?field.
           ?field rdfs:label ?field_label.}
  OPTIONAL{?paper orkgp:P27 ?author.
           BIND(IF(isLiteral(?author), ?author, "") AS ?name1)
           OPTIONAL{?author rdfs:label ?author_label;
                            orkgp:HAS_ORCID ?id.}
           BIND(IF(BOUND(?author_label),?author_label, "") AS ?name2)
           BIND(IF(?name1 = "", ?name2, ?name1) AS ?name)
          }
  
  OPTIONAL{?paper orkgp:P28 ?month.
           BIND(IF(isLiteral(?month), ?month, "") AS ?month1)
           OPTIONAL{?month rdfs:label ?month_label.}
           BIND(IF(BOUND(?month_label),?month_label, "") AS ?month2)
           BIND(IF(?month1 = "", ?month2, ?month1) AS ?month_number)
          }
  
  OPTIONAL{?paper orkgp:P29 ?year.
           BIND(IF(isLiteral(?year), ?year, "") AS ?year1)
           OPTIONAL{?year rdfs:label ?year_label.}
           BIND(IF(BOUND(?year_label),?year_label, "") AS ?year2)
           BIND(IF(?year1 = "", ?year2, ?year1) AS ?year_number)
          }
  
  OPTIONAL{?paper orkgp:url ?paper_url.}
  OPTIONAL{?paper orkgp:HAS_VENUE ?venue.
           ?venue rdfs:label ?venue_label.}
} ORDER BY ?paper
        """

user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])

sparql = SPARQLWrapper(ENDPOINT_URL, agent=user_agent)
sparql.setQuery(PREFIXES+query)
sparql.setReturnFormat(CSV)

try:
        results = sparql.queryAndConvert()
except Exception as e:
        print(e)

now = datetime.now()
with open('query_result_' + now.strftime('%Y-%m-%d') + '.csv', 'wb') as file:
        file.write(results)

#df = pd.read_csv('CLEAN-query-a_paper_and_all_metadata_optional_result_1-41364-2022.05.04.csv')
md_df = pd.read_csv('query_result_' + now.strftime('%Y-%m-%d') + '.csv', encoding='utf-8', encoding_errors='ignore')

print(md_df['paper'].nunique())

9880


## Data Cleaning

In [112]:
md_df[md_df['month_number'] != '10']

,paper,field_label,DOI,title,name,id,month_number,year_number,paper_url,venue_label,field,author,orcid
0,http://orkg.org/orkg/resource/R1000,Computational Engineering,nan,nan,NaN,NaN,1,2000.0,NaN,NaN,Computational Engineering,nan,nan
3,http://orkg.org/orkg/resource/R10024,Engineering,10.1016/j.trpro.2018.10.057,Reporting quality of travel and non-travel act...,Reinhard Hössinger,NaN,NaN,2018.0,NaN,Transportation Research Procedia,Engineering,Reinhard Hössinger,nan
4,http://orkg.org/orkg/resource/R10024,Engineering,10.1016/j.trpro.2018.10.057,Reporting quality of travel and non-travel act...,Regine Gerike,NaN,NaN,2018.0,NaN,Transportation Research Procedia,Engineering,Regine Gerike,nan
5,http://orkg.org/orkg/resource/R10024,Engineering,10.1016/j.trpro.2018.10.057,Reporting quality of travel and non-travel act...,Florian Aschauer,NaN,NaN,2018.0,NaN,Transportation Research Procedia,Engineering,Florian Aschauer,nan
6,http://orkg.org/orkg/resource/R10024,Engineering,10.1016/j.trpro.2018.10.057,Reporting quality of travel and non-travel act...,Basil Schmid,NaN,NaN,2018.0,NaN,Transportation Research Procedia,Engineering,Basil Schmid,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41359,http://orkg.org/orkg/resource/R9567,Artificial Intelligence,10.1109/icdar.2019.00231,A Neural Approach for Text Extraction from Sch...,Ralph Ewerth,0000-0003-0918-6297,9,2019.0,NaN,2019 International Conference on Document Ana...,Artificial Intelligence,Ralph Ewerth,0000-0003-0918-6297
41360,http://orkg.org/orkg/resource/R9567,Artificial Intelligence,10.1109/icdar.2019.00231,A Neural Approach for Text Extraction from Sch...,Peichen Tang,NaN,9,2019.0,NaN,2019 International Conference on Document Ana...,Artificial Intelligence,Peichen Tang,nan
41361,http://orkg.org/orkg/resource/R9639,Computer Sciences,10.1007/978-3-030-30760-8_18,A Human-Friendly Query Generation Frontend for...,Said Fathalla,0000-0002-2818-5890,NaN,2019.0,NaN,NaN,Computer Sciences,Said Fathalla,0000-0002-2818-5890
41362,http://orkg.org/orkg/resource/R9639,Computer Sciences,10.1007/978-3-030-30760-8_18,A Human-Friendly Query Generation Frontend for...,Sören Auer,0000-0002-0698-2864,NaN,2019.0,NaN,NaN,Computer Sciences,Sören Auer,0000-0002-0698-2864


In [109]:
df = pd.DataFrame()

df['paper'] = md_df['paper'].astype('str')
df['field'] = md_df['field_label'].astype('category')
df['DOI'] = md_df['DOI'].astype('str')
df['title'] = md_df['title'].astype('str')
df['author'] = md_df['name'].astype('str')
df['orcid'] = md_df['id'].astype('str')
df['month'] = md_df['month_number'].fillna(0.0).astype('int')
df['year'] = md_df['year_number'].fillna(0.0).astype('int')
df['url'] = md_df['paper_url'].astype('str')
df['venue'] = md_df['venue_label'].astype('str')

field_df = md_df[['paper','field_label']].drop_duplicates()
print(field_df.isnull().sum())

field_df.fillna('No research field')
field_df[['field_label']].value_counts().head(15)

ValueError: invalid literal for int() with base 10: 'October 2020'